In [1]:
import sys
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
# need ms coco api to get dataset
from pycocotools import mask as mask
import numpy as np
import matplotlib.pyplot as plt
sys.path.insert(0, "../utils")
import utils
import train_eval

In [2]:
utils.set_random_seed()

In [3]:
import coco_dataset
import importlib
importlib.reload(coco_dataset)

# path to MS COCO dataset
train_data_dir = '/home/nfs/inf6/data/datasets/coco/train2017'
val_data_dir = '/home/nfs/inf6/data/datasets/coco/val2017'


# initialize COCO API for segmentation
train_ann_file = '/home/nfs/inf6/data/datasets/coco/annotations/instances_train2017.json'
val_ann_file = '/home/nfs/inf6/data/datasets/coco/annotations/instances_val2017.json'


# make dataset, default transforms is resizing and to tensor
train_set = coco_dataset.Coco_Dataset(train_data_dir, train_ann_file, mode="segmentation")
val_set = coco_dataset.Coco_Dataset(val_data_dir, val_ann_file, mode="segmentation")

batch_size = 16
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

loading annotations into memory...
Done (t=14.60s)
creating index...
index created!
loading annotations into memory...
Done (t=0.47s)
creating index...
index created!


In [4]:
# img, seg_mask = train_set[6]

# _, ax = plt.subplots(1, 2)

# ax[0].imshow(img.permute(1, 2, 0), cmap="gray")
# ax[1].imshow(seg_mask.squeeze(), cmap="gray")

# _, x = plt.subplots(1, len(masks), figsize=(20,20))
# for i in range(len(masks)):
#     mask = masks[i]
#     idx = mask == 0
#     mask[mask == 0] = 100
#     x[i].imshow(mask.squeeze(), cmap="gray")



In [16]:
import UNet_Template
import importlib
importlib.reload(UNet_Template)

config = UNet_Template.RNN_UNetConfig()

rnn_unet = UNet_Template.RNN_UNet(config)

rnn_unet_optim = torch.optim.Adam(rnn_unet.parameters(), lr=3e-4)

criterion = nn.CrossEntropyLoss()

In [17]:
importlib.reload(train_eval)

epochs=10
rnn_unet_trainer = train_eval.Trainer(
    rnn_unet, rnn_unet_optim, criterion, train_loader, valid_loader, epochs, sequence=False)

/home/user/schwemme/CudaLab_Project/src/tboard_logs/RNN_UNet


In [18]:
rnn_unet_trainer.train_model()

tensor([ 0.,  1.,  3.,  7.,  8.,  9., 13., 16., 23., 27., 31., 35., 40., 42.,
        43., 51., 54., 62., 64., 65., 67., 72., 73., 74., 76., 77., 78., 79.,
        82., 84., 85., 86., 88.], device='cuda:0')
torch.float32 torch.float32
tensor(17452.6250, device='cuda:0')
torch.Size([])
tensor([ 0.,  1.,  3.,  5.,  6.,  8., 10., 17., 24., 32., 35., 39., 42., 46.,
        47., 49., 51., 52., 53., 60., 62., 65., 67., 76.], device='cuda:0')
torch.float32 torch.float32
tensor(21213.9492, device='cuda:0')
torch.Size([])
tensor([ 0.,  1.,  2.,  9., 16., 17., 27., 31., 32., 35., 41., 42., 44., 46.,
        47., 48., 49., 50., 51., 52., 56., 57., 62., 63., 67., 70., 72., 73.,
        75., 76., 81., 87.], device='cuda:0')
torch.float32 torch.float32
tensor(25107.4102, device='cuda:0')
torch.Size([])
tensor([ 0.,  1.,  3.,  5.,  6.,  7.,  8., 15., 20., 35., 39., 40., 42., 43.,
        44., 50., 51., 52., 53., 55., 70., 81.], device='cuda:0')
torch.float32 torch.float32
tensor(8592.4062, device='cu

KeyboardInterrupt: 

In [ ]:
city_home = "/home/nfs/inf6/data/datasets/cityscapes"

cityscapes_train = torchvision.datasets.Cityscapes(root=city_home, split="train", mode="coarse", target_type="semantic")
cityscapes_test = torchvision.datasets.Cityscapes(root=city_home, split="train_extra", mode="coarse", target_type="semantic")
cityscapes_valid = torchvision.datasets.Cityscapes(root=city_home, split="val", mode="coarse", target_type="semantic")


In [14]:
img, label_dict = train_set[6]
masks = label_dict["masks"]
labels = label_dict["labels"]
seg_mask = label_dict["segmentation_mask"]

_, ax = plt.subplots(1, 2)

ax[0].imshow(img.permute(1, 2, 0), cmap="gray")
ax[1].imshow(seg_mask.squeeze(), cmap="gray")

_, x = plt.subplots(1, len(masks), figsize=(20,20))
for i in range(len(masks)):
    mask = masks[i]
    idx = mask == 0
    mask[mask == 0] = 100
    x[i].imshow(mask, cmap="gray")



uint8
<class 'numpy.ndarray'>


TypeError: Unexpected type <class 'numpy.ndarray'>